In [80]:
import matplotlib as plt
import pandas as pd
import os

def filter(df,column, value):
    indexNames = df[ (df[column] != value)].index
    df.drop(indexNames , inplace=True)
    df = df.reset_index(drop=True)
    return df

def getInfoFromFilename(file):
        tokenList = statFiles[0].split(' - ')
        tokenList[2] = tokenList[2][:-10]
        tokenList.pop(1)
        return tokenList

In [68]:
class Scenario:
    name = None
    scores = {}
    
    def __init__(self, name):
        self.name = name
    
    def addScoreFromFilename(timestamp, score):
        # TODO: filter out bad formatting
        scores[timestamp] = score

In [81]:
STATS_DIRECTORY = 'C:\ProgramData\SteamLibrary\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats'

statFiles = []
for root, _, files in os.walk(STATS_DIRECTORY):
    for file in files:
        if file.endswith('.csv'):
            statFiles.append(file)
            
print(statFiles[0])

df = pd.read_csv(STATS_DIRECTORY + '\\' +  statFiles[0], sep=',', error_bad_lines=False)

df.rename(columns={'Kill #':'Headings'}, inplace=True)
df.rename(columns={'Timestamp':'Values'}, inplace=True)

df = df[['Headings','Values']]

df = filter(df, 'Headings', 'Score:')

df
# print(df['Values'][103])

name, timestamp = getInfoFromFilename(statFiles[0])
print(name)
print(timestamp)


1wall 1target - Challenge - 2020.05.15-12.25.26 Stats.csv
1wall 1target
2020.05.15-12.25.26
